In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow.keras as K
import numpy as np
import pandas as pd 
import cv2
import os
import matplotlib.pyplot as mpl
import pandas as pd
import numpy as np
from tensorflow.keras.saving import load_model
import tensorflow as tf
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from tensorflow.keras import Sequential


v16_model = VGG16(weights="imagenet", include_top=False, input_shape=(32,32,3))


for i in v16_model.layers:
    i.trainable = False  


def invert_threshold(image, threshold=0.5):
    inverted_image = tf.where(image > threshold, 0.0, 1.0)  
    return inverted_image


train_dir = '/kaggle/input/gujarati-p-data/gujarati-handwritten-digit-dataset-master/Train Set'
test_dir = '/kaggle/input/gujarati-p-data/gujarati-handwritten-digit-dataset-master/Test Set'


train_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: invert_threshold(x / 255.0)
)

test_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: invert_threshold(x / 255.0)
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),  
    color_mode='grayscale',  
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),  
    color_mode='grayscale',  
    batch_size=32,
    class_mode='categorical'
)


model = Sequential()


model.add(layers.Conv2D(3, (3, 3), padding='same', input_shape=(32, 32, 1)))


model.add(v16_model)


model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])


model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    callbacks=[early_stopping]
)


test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


In [3]:
model.save('/kaggle/working/vggsf16.keras')